## Notebook for making predictions with Normalized train data


In [13]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [14]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [15]:
X_A_train = dm.data_A_obs.iloc[:, 2:-1]
y_A_train = dm.data_A_obs.iloc[:,0]
X_B_train = dm.data_B_obs.iloc[:, 2:-1]
y_B_train = dm.data_B_obs.iloc[:,0]
X_C_train = dm.data_C_obs.iloc[:, 2:-1]
y_C_train = dm.data_C_obs.iloc[:,0]

X_A_val = dm.data_A_es.iloc[:, 2:-1]
y_A_val = dm.data_A_es.iloc[:,0]
X_B_val = dm.data_B_es.iloc[:, 2:-1]
y_B_val = dm.data_B_es.iloc[:,0]

X_C_val = dm.data_C_es.iloc[:, 2:-1]
y_C_val = dm.data_C_es.iloc[:,0]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_b.columns.intersection(X_B_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_c.columns.intersection(X_C_train.columns)]

In [47]:
# Scale down y values by mean
y_A_train /= dm.amplitude[0]
y_B_train /= dm.amplitude[1]
y_C_train /= dm.amplitude[2]

y_A_val /= dm.amplitude[0]
y_B_val /= dm.amplitude[1]
y_C_val /= dm.amplitude[2]

In [50]:
y_B_val.describe()

count    14981.000000
mean         0.507965
std          1.405032
min          0.000000
25%          0.000000
50%          0.000000
75%          0.170600
max          8.934363
Name: pv_measurement, dtype: float64

In [16]:
# Concat

X_train = pd.concat([X_A_train,X_B_train,X_C_train])
y_train = pd.concat([y_A_train,y_B_train,y_C_train])

X_val = pd.concat([X_A_val,X_B_val,X_C_val])
y_val = pd.concat([y_A_val,y_B_val,y_C_val])

#X_test = pd.concat([X_A_test,X_B_test,X_C_test])

## Catboost models

In [52]:
catModel = CatBoostRegressor()
y_train.describe()

count    328774.000000
mean          1.065819
std           2.031228
min           0.000000
25%           0.000000
50%           0.002620
75%           1.159637
max          12.980092
Name: pv_measurement, dtype: float64

In [17]:
print(X_train.shape)
print(y_train.shape)

X_train.head(2)



(328774, 25)
(328774,)


,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,...,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_w_1000hPa:ms,month,hours,sum_rad:W
0,7.7,1744.900024,1744.900024,0.0,280.299988,0.0,0.0,1.0,1006.799988,0.0,...,-0.0,0.1,-3.202,100.0,39640.101562,3.7,-0.0,6,22,0.0
1,7.7,1734.000000,1734.000000,0.0,280.299988,0.0,0.0,1.0,1006.500000,0.0,...,-0.0,0.2,-3.650,100.0,40123.898438,3.6,-0.0,6,22,0.0


In [53]:
catModel.fit(X_train,y_train)

Learning rate set to 0.102294
0:	learn: 1.8782779	total: 26.1ms	remaining: 26s
1:	learn: 1.7424063	total: 40.4ms	remaining: 20.2s
2:	learn: 1.6219531	total: 53.8ms	remaining: 17.9s
3:	learn: 1.5161447	total: 66.5ms	remaining: 16.6s
4:	learn: 1.4254765	total: 78.4ms	remaining: 15.6s
5:	learn: 1.3465396	total: 89.4ms	remaining: 14.8s
6:	learn: 1.2769850	total: 101ms	remaining: 14.3s
7:	learn: 1.2166926	total: 113ms	remaining: 14s
8:	learn: 1.1657351	total: 124ms	remaining: 13.6s
9:	learn: 1.1223995	total: 136ms	remaining: 13.5s
10:	learn: 1.0841526	total: 148ms	remaining: 13.3s
11:	learn: 1.0516389	total: 162ms	remaining: 13.3s
12:	learn: 1.0242590	total: 176ms	remaining: 13.4s
13:	learn: 1.0008748	total: 189ms	remaining: 13.3s
14:	learn: 0.9812649	total: 200ms	remaining: 13.2s
15:	learn: 0.9636988	total: 216ms	remaining: 13.3s
16:	learn: 0.9496362	total: 230ms	remaining: 13.3s
17:	learn: 0.9372403	total: 240ms	remaining: 13.1s
18:	learn: 0.9266148	total: 251ms	remaining: 13s
19:	learn: 

In [54]:
score = catModel.score(X_val, y_val)

print(score)

0.8007656453493568


## Preds

In [55]:
pred_A = catModel.predict(X_A_test)*dm.amplitude[0]
pred_B = catModel.predict(X_B_test)*dm.amplitude[1]
pred_C = catModel.predict(X_C_test)*dm.amplitude[2]



In [56]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

# NAME THE FILE 
df.to_csv("sub32.csv")

df.shape

(2160, 1)